# <font color='#eb3483'> Import Data </font>

In [41]:
#from scipy import stats
import numpy as np
import pandas as pd
import seaborn as sns

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [42]:
df = pd.read_csv("compas.csv", skip_blank_lines = True)
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,8/14/13,Male,4/18/47,69,Greater than 45,Other,...,1,Low,8/14/13,7/7/14,7/14/14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,1/27/13,Male,1/22/82,34,25 - 45,African-American,...,1,Low,1/27/13,1/26/13,2/5/13,0,9,159,1,1
2,4,ed philo,ed,philo,4/14/13,Male,5/14/91,24,Less than 25,African-American,...,3,Low,4/14/13,6/16/13,6/16/13,4,0,63,0,1
3,5,marcu brown,marcu,brown,1/13/13,Male,1/21/93,23,Less than 25,African-American,...,6,Medium,1/13/13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,3/26/13,Male,1/22/73,43,25 - 45,Other,...,1,Low,3/26/13,NaN,NaN,2,0,1102,0,0


In [43]:
df.shape

(7214, 53)

In [44]:
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [45]:
df.info

<bound method DataFrame.info of          id                 name      first         last  \
0         1     miguel hernandez     miguel    hernandez   
1         3          kevon dixon      kevon        dixon   
2         4             ed philo         ed        philo   
3         5          marcu brown      marcu        brown   
4         6   bouthy pierrelouis     bouthy  pierrelouis   
...     ...                  ...        ...          ...   
7209  10996        steven butler     steven       butler   
7210  10997      malcolm simmons    malcolm      simmons   
7211  10999      winston gregory    winston      gregory   
7212  11000          farrah jean     farrah         jean   
7213  11001  florencia sanmartin  florencia    sanmartin   

     compas_screening_date     sex       dob  age          age_cat  \
0                  8/14/13    Male   4/18/47   69  Greater than 45   
1                  1/27/13    Male   1/22/82   34          25 - 45   
2                  4/14/13    Male   

In [46]:
df = df.drop(['name', 'first', 'last'], axis=1)
df.head()

,id,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,8/14/13,Male,4/18/47,69,Greater than 45,Other,0,1,0,...,1,Low,8/14/13,7/7/14,7/14/14,0,0,327,0,0
1,3,1/27/13,Male,1/22/82,34,25 - 45,African-American,0,3,0,...,1,Low,1/27/13,1/26/13,2/5/13,0,9,159,1,1
2,4,4/14/13,Male,5/14/91,24,Less than 25,African-American,0,4,0,...,3,Low,4/14/13,6/16/13,6/16/13,4,0,63,0,1
3,5,1/13/13,Male,1/21/93,23,Less than 25,African-American,0,8,1,...,6,Medium,1/13/13,NaN,NaN,1,0,1174,0,0
4,6,3/26/13,Male,1/22/73,43,25 - 45,Other,0,1,0,...,1,Low,3/26/13,NaN,NaN,2,0,1102,0,0


# <font color='#eb3483'> Data Analysis </font>

Questions: Does the algorithm lead to biased results?

Clean data after break them down according to topics for convienice, when cleaning, check for:
1. missing values
2. duplicates
3. cardinality (how many repetitive values)
4. outliners

Steps for analysis:
1. pick out columns, clean & prepare data, raname columns
2. run analysis
3. make graphs

## <font color='#eb3483'> 1.  </font>